# 빅데이터 분석 기사 실기 스터디 2주차 과제

***

## Machine Learning - 5문제

### 1. 영화 관객 수(box_off_num) 예측 모델을 만들어 학습한 후 RMSE를 확인해보세요.

- 데이터 파일명은 movies_train 입니다.
- 데이터를 7:3 비율로 나누고 validation 데이터에 대해 RMSE를 출력하세요.
- 데이터 전부 사용

In [15]:
import pandas as pd

In [41]:
data = pd.read_csv("/Users/kisehyun/Competition/데이콘_영화관객예측/movies_train.csv")

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [43]:
data.fillna(0, inplace = True)

In [44]:
data.isnull().sum()

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
box_off_num       0
dtype: int64

In [45]:
data.drop(['title', 'distributor', 'director'], axis = 1, inplace = True)

In [46]:
data['날짜'] = pd.to_datetime(data['release_time'])

In [47]:
data['월'] = data['날짜'].dt.month

In [48]:
data

,genre,release_time,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,날짜,월
0,액션,2012-11-22,96,청소년 관람불가,0.00,0,91,2,23398,2012-11-22,11
1,느와르,2015-11-19,130,청소년 관람불가,1161602.50,2,387,3,7072501,2015-11-19,11
2,액션,2013-06-05,123,15세 관람가,220775.25,4,343,4,6959083,2013-06-05,6
3,코미디,2012-07-12,101,전체 관람가,23894.00,2,20,6,217866,2012-07-12,7
4,코미디,2010-11-04,108,15세 관람가,1.00,1,251,2,483387,2010-11-04,11
...,...,...,...,...,...,...,...,...,...,...,...
595,드라마,2014-08-13,111,청소년 관람불가,3833.00,1,510,7,1475091,2014-08-13,8
596,드라마,2013-03-14,127,15세 관람가,496061.00,1,286,6,1716438,2013-03-14,3
597,공포,2010-09-30,99,청소년 관람불가,0.00,0,123,4,2475,2010-09-30,9
598,느와르,2015-05-14,102,15세 관람가,0.00,0,431,4,2192525,2015-05-14,5


In [49]:
rank_genre = data.groupby('genre').box_off_num.median().sort_values().reset_index()

In [50]:
rank_genre['g_ranking'] = [i for i in range(1,13)]

In [51]:
rank_genre

,genre,box_off_num,g_ranking
0,다큐멘터리,3044.0,1
1,미스터리,3199.0,2
2,멜로/로맨스,3653.0,3
3,뮤지컬,4778.0,4
4,드라마,7173.0,5
5,SF,9323.0,6
6,애니메이션,23802.0,7
7,서스펜스,82611.0,8
8,공포,179810.0,9
9,코미디,210775.0,10


In [52]:
data = pd.merge(data, rank_genre.drop('box_off_num', axis = 1), how = 'left')

In [53]:
data.head()

,genre,release_time,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,날짜,월,g_ranking
0,액션,2012-11-22,96,청소년 관람불가,0.00,0,91,2,23398,2012-11-22,11,11
1,느와르,2015-11-19,130,청소년 관람불가,1161602.50,2,387,3,7072501,2015-11-19,11,12
2,액션,2013-06-05,123,15세 관람가,220775.25,4,343,4,6959083,2013-06-05,6,11
3,코미디,2012-07-12,101,전체 관람가,23894.00,2,20,6,217866,2012-07-12,7,10
4,코미디,2010-11-04,108,15세 관람가,1.00,1,251,2,483387,2010-11-04,11,10


In [54]:
data.groupby('screening_rat').box_off_num.median()

screening_rat
12세 관람가      19323.0
15세 관람가     242381.0
전체 관람가        8325.0
청소년 관람불가      3008.5
Name: box_off_num, dtype: float64

In [55]:
data['screening_rat'] = data['screening_rat'].map({'12세 관람가' : 3, '전체 관람가' : 2, '청소년 관람불가' : 1, '15세 관람가' : 4})

In [56]:
X = data.drop(['genre', 'release_time', '날짜'], axis =1)

In [57]:
X

,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,월,g_ranking
0,96,1,0.00,0,91,2,23398,11,11
1,130,1,1161602.50,2,387,3,7072501,11,12
2,123,4,220775.25,4,343,4,6959083,6,11
3,101,2,23894.00,2,20,6,217866,7,10
4,108,4,1.00,1,251,2,483387,11,10
...,...,...,...,...,...,...,...,...,...
595,111,1,3833.00,1,510,7,1475091,8,5
596,127,4,496061.00,1,286,6,1716438,3,5
597,99,1,0.00,0,123,4,2475,9,9
598,102,4,0.00,0,431,4,2192525,5,12


In [58]:
y = data['box_off_num']

In [59]:
y

0        23398
1      7072501
2      6959083
3       217866
4       483387
        ...   
595    1475091
596    1716438
597       2475
598    2192525
599    7166532
Name: box_off_num, Length: 600, dtype: int64

In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [61]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [62]:
rf = RandomForestRegressor(random_state = 42)

In [63]:
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [64]:
rf_pred = rf.predict(X_val)

In [65]:
from sklearn.metrics import mean_squared_error

In [66]:
rmse = mean_squared_error(y_val, rf_pred) ** 0.5

In [67]:
rmse

31947.499000787986

### 2. 보험 가입 여부(Response) 예측 모델을 만들어 학습한 후 정확도를 출력해보세요.
- 데이터 파일명은 health_train입니다.
- 데이터를 8:2로 나누고 validation 데이터에 대해 정확도를 출력하세요.